In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from random import *

In [ ]:
def prob(y): #probability of reward
    if random() <= y:
        return 1
    else:
        return 0

In [ ]:
def trialgenerator(y): #generating 1000 trials for the two levers; the levers switch probabilities every 10-32 trials
    left_reward = []
    right_reward = []
    left_correct = []
    right_correct = []
    for trials in np.arange(50):
        for x in np.arange(randint(10,32)):
            left_reward.append(prob(y))
            right_reward.append(prob(1-y))
            left_correct.append(1)   #correct is 1 when reward probability is higher and 0 when lower
            right_correct.append(0)
        for x in np.arange(randint(10,32)):
            left_reward.append(prob(1-y))
            right_reward.append(prob(y))
            left_correct.append(0)
            right_correct.append(1)
    del left_correct [1000:] #deleting everything after 1000 trials
    del right_correct [1000:]
    del left_reward[1000:]
    del right_reward[1000:]
    return left_reward, right_reward, left_correct, right_correct

In [ ]:
def lever_update(alpha, value, reward): 
    return value + alpha *(reward - value) 

In [ ]:
def softmax(beta, temp_value1, temp_value2):   #temp_value1 is for left lever and temp_value2 is for right lever
    num = np.exp(temp_value1 * beta)
    den = np.exp(temp_value1 * beta) + np.exp(temp_value2 * beta)    
    return num / den

In [ ]:
def sim(alpha, beta, left_reward, right_reward, left_correct, right_correct): #bandit task simulated
    right_value = [0.5]
    left_value = [0.5]
    correct = []
    for index, lr in enumerate(left_reward):
        if random() <= softmax(beta, left_value[-1], right_value[-1]):
            left_value.append(lever_update(alpha, left_value[-1], left_reward[index]))
            right_value.append(right_value[-1])
            correct.append(left_correct[index])
        else:
            right_value.append(lever_update(alpha, right_value[-1], right_reward[index]))
            left_value.append(left_value[-1])
            correct.append(right_correct[index])
    return correct

In [ ]:
correct_choices = {} 

def figure2(prob):  #used to create data for 2c heatmaps
    for x in np.arange(0,1.05,0.05): #range of alphas 0 to 1
        x = round(x, 3)
        correct_choices[x] = {}
        for y in np.arange(0,5.1,0.1): #range of beta from 0 to 5
            y = round(y, 3)
            correct_choices[x][y] = {}
            correct_list = []
            for n in range(1):
                left_reward, right_reward, left_correct, right_correct = trialgenerator(prob)  #setting reward for left lever
                correct_list.append(sim(x, y, left_reward, right_reward, left_correct, right_correct))
            correct_choices[x][y] = correct_list
    return correct_choices[x][y]